In [1678]:
# jupyter nbconvert --to python Extract_Playlist_With_Duplicates.ipynb

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import csv
import pandas as pd
from datetime import datetime

In [1679]:
print("...Making CSV...")

...Making CSV...


In [1680]:
#Authentication - without user
with open(r"C:\Users\mackt\Python\Music Library\Spotify_App_Credentials.txt") as f:
    sac_lines = f.readlines()
    cid = sac_lines[0].split(", ")
    cid = cid[1].split("\n")
    cid = cid[0]
    # print(cid)
    secret = sac_lines[1].split(", ")
    secret = secret[1]
    # print(secret)

In [1681]:
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [1682]:
# playlist_link = "https://open.spotify.com/playlist/5F5yHyXHt6vp2taA7DrEhJ?si=b4f82e9651444f28"
playlist_link = "https://open.spotify.com/playlist/7LaicnuGlBjUoHZ5Rd4tjm?si=e9f47ebd992b4d08"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]

In [1683]:
results = sp.playlist_tracks(playlist_URI)
tracks = results['items']
while results['next']:
    results = sp.next(results)
    tracks.extend(results['items'])

In [1684]:
for track in tracks:
    # print(track["track"]["name"])
    pass
# print(sp.audio_analysis(track["track"]["id"]), '\n')
print(sp.audio_features(track["track"]["uri"]), '\n')
print(sp.track(track["track"]["uri"]), '\n')
print(track["track"]["artists"][0]["id"], '\n')
print(sp.artist(track["track"]["artists"][0]["id"])["genres"], '\n')


[{'danceability': 0.73, 'energy': 0.777, 'key': 3, 'loudness': -5.199, 'mode': 0, 'speechiness': 0.0479, 'acousticness': 0.108, 'instrumentalness': 0.000467, 'liveness': 0.069, 'valence': 0.591, 'tempo': 130.0, 'type': 'audio_features', 'id': '20zQZVyUNPbq8kZACdgYrh', 'uri': 'spotify:track:20zQZVyUNPbq8kZACdgYrh', 'track_href': 'https://api.spotify.com/v1/tracks/20zQZVyUNPbq8kZACdgYrh', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/20zQZVyUNPbq8kZACdgYrh', 'duration_ms': 284867, 'time_signature': 4}] 

{'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2DlGxzQSjYe5N6G9nkYghR'}, 'href': 'https://api.spotify.com/v1/artists/2DlGxzQSjYe5N6G9nkYghR', 'id': '2DlGxzQSjYe5N6G9nkYghR', 'name': 'Jennifer Lopez', 'type': 'artist', 'uri': 'spotify:artist:2DlGxzQSjYe5N6G9nkYghR'}], 'available_markets': ['CA', 'MX', 'US'], 'external_urls': {'spotify': 'https://open.spotify.com/album/1RJ3ZqXAzt0rsJhs80AqRb'}, 'href': 'https://api.

In [1685]:
# Song Library

sl_tr_id = []



sl_tr_uri1 = []
sl_tr_uri2 = []
sl_tr_uri3 = []
sl_tr_uri4 = []
sl_tr_uri5 = []

Duplicate_Index = []
Duplicate_ID = []

sl_tr_name = []
sl_tr_name_LOW = []
# All uris for each song
sl_tr_uri = []

# All Artists for each song
sl_ar_name = []
sl_ar_uri = []

lib = {}
l = 0
track_id = 1

for track in tracks:
    lib["track{0}". format(l)] = {}

    # Track Name and URI
    lib["track{0}". format(l)]["id"] = l

    # Track Name and URI
    track_uri = track["track"]["uri"]
    track_name = track["track"]["name"]

    lib["track{0}". format(l)]["uri"] = track_uri
    lib["track{0}". format(l)]["name"] = track_name
    lib["track{0}". format(l)]["name_low"] = track_name.lower()

    sl_tr_name.append(track_name)
    sl_tr_name_LOW.append(track_name.lower())
    sl_tr_uri.append(track_uri)

    # Artists Name and URI
    track_artists_name = []
    track_artists_uri = []

    lib["track{0}". format(l)]["artists"] = {}
    for i in range (10):
        lib["track{0}". format(l)]["artists"]["ar_name{0}". format(i)] = track["track"]["artists"][i]["name"]
        lib["track{0}". format(l)]["artists"]["ar_uri{0}". format(i)] = track["track"]["artists"][i]["uri"]

        track_artists_name.append(track["track"]["artists"][i]["name"])
        track_artists_uri.append(track["track"]["artists"][i]["uri"])

        i = i + 1
        try: track["track"]["artists"][i]["name"]
        except IndexError:
            y = i-1
            sl_ar_name.append(track_artists_name)
            sl_ar_uri.append(track_artists_uri)
            break

    # Track Release Date + Local
    # Local Tracks
    if track['is_local'] == True:
        lib["track{0}". format(l)]["date_spot"] = 'None'
        lib["track{0}". format(l)]["local"] = 'Yes'
    # Spotify Tracks
    else:
        track_release_date = sp.track(track_uri)["album"]["release_date"]
        if sp.track(track_uri)["album"]["release_date_precision"] == 'day':
            track_release_date = datetime.strptime(track_release_date, "%Y-%m-%d").date()
        elif sp.track(track_uri)["album"]["release_date_precision"] == 'month':
            track_release_date = datetime.strptime(track_release_date, "%Y-%m").date()
        elif sp.track(track_uri)["album"]["release_date_precision"] == 'year':
            track_release_date = datetime.strptime(track_release_date, "%Y").date()
        lib["track{0}". format(l)]["date_spot"] = track_release_date
        lib["track{0}". format(l)]["local"] = 'No'

    # Track Duration
    feature_aud = sp.audio_features(track_uri)[0]

    if track['is_local'] == True:
        lib["track{0}". format(l)]["time"] = track["track"]["duration_ms"]
    else:
        lib["track{0}". format(l)]["time"] = feature_aud["duration_ms"]

    # Track Album
    lib["track{0}". format(l)]["album"] = track["track"]["album"]["name"]

    # Track Popularity
    lib["track{0}". format(l)]["popularity"] = track["track"]["popularity"]

    # Track Audio Features
    # Locals
    if track['is_local'] == True:
        lib["track{0}". format(l)]["danceability"] = 'None'
        lib["track{0}". format(l)]["energy"] = 'None'
        lib["track{0}". format(l)]["key"] = 'None'
        lib["track{0}". format(l)]["loudness"] = 'None'
        lib["track{0}". format(l)]["mode"] = 'None'
        lib["track{0}". format(l)]["speechiness"] = 'None'
        lib["track{0}". format(l)]["acousticness"] = 'None'
        lib["track{0}". format(l)]["instrumentalness"] = 'None'
        lib["track{0}". format(l)]["liveness"] = 'None'
        lib["track{0}". format(l)]["valence"] = 'None'
        lib["track{0}". format(l)]["tempo"] = 'None'
    # Spotify
    else:
        lib["track{0}". format(l)]["danceability"] = feature_aud["danceability"]
        lib["track{0}". format(l)]["energy"] = feature_aud["energy"]
        lib["track{0}". format(l)]["key"] = feature_aud["key"]
        lib["track{0}". format(l)]["loudness"] = feature_aud["loudness"]
        lib["track{0}". format(l)]["mode"] = feature_aud["mode"]
        lib["track{0}". format(l)]["speechiness"] = feature_aud["speechiness"]
        lib["track{0}". format(l)]["acousticness"] = feature_aud["acousticness"]
        lib["track{0}". format(l)]["instrumentalness"] = feature_aud["instrumentalness"]
        lib["track{0}". format(l)]["liveness"] = feature_aud["liveness"]
        lib["track{0}". format(l)]["valence"] = feature_aud["valence"]
        lib["track{0}". format(l)]["tempo"] = feature_aud["tempo"]

    # Duplicate Songs
    lib["track{0}". format(l)]["duplicates"] = {}
    lib["track{0}". format(l)]["duplicates"]["dup_id"] = -1
    lib["track{0}". format(l)]["duplicates"]["num_dup"] = 0

    track_uris = [track_uri]
    current_index = len(lib)

    print(track_name)
    same_tr_names = sl_tr_name_LOW.count(track_name.lower())-1
    dup = 0
    if same_tr_names != 0:
        last_ind = 0
        # print(same_tr_names, "same name")
        for i in range(same_tr_names):
            index = sl_tr_name_LOW.index(track_name.lower(), last_ind)
            if sl_ar_uri[index] == track_artists_uri:
                if dup == 0:
                    dup = 1
                    dup_id = index
                    Duplicate_Index.append(dup_id)
                    Duplicate_Index[index] = dup_id

                    lib["track{0}". format(l)]["duplicates"]["num_dup"] = dup
                    lib["track{0}". format(index)]["duplicates"]["num_dup"] = dup

                    lib["track{0}". format(l)]["duplicates"]["dup_id"] = dup_id
                    lib["track{0}". format(index)]["duplicates"]["dup_id"] = dup_id

                    lib["track{0}". format(index)]["duplicates"]["dup_loc{0}".format(dup-1)] = index

                else:
                    lib["track{0}". format(l)]["duplicates"]["num_dup"] = dup

                lib["track{0}". format(l)]["duplicates"]["dup_loc{0}".format(dup-1)] = index
                lib["track{0}". format(l)]["duplicates"]["dup_loc{0}".format(dup)] = l

                # iter = dup
                # for x in range(dup):
                #     prev_tr = lib["track{0}". format(l)]["duplicates"]["dup_loc{0}".format(x)]
                #     lib["track{0}". format(prev_tr)]["duplicates"]["num_dup"] = dup
                #     iter = iter - x

                #     for y in range (iter):
                #         a = lib["track{0}". format(l)]["duplicates"]["dup_loc{0}".format(x+y+1)]
                #         lib["track{0}". format(prev_tr)]["duplicates"]["dup_loc{0}".format(x+y+1)] = a

                dup = dup + 1
                # print(index, "+")
            elif i == same_tr_names-1 and dup == 0:
                Duplicate_Index.append(-1)
                # print(index, "-")

            last_ind = index+1

    else:
        Duplicate_Index.append(-1)

    dup = dup - 1
    iter = dup
    for x in range(dup):
        prev_tr = lib["track{0}". format(l)]["duplicates"]["dup_loc{0}".format(x)]
        lib["track{0}". format(prev_tr)]["duplicates"]["num_dup"] = dup
        iter = iter - x

        for y in range (iter):
            a = lib["track{0}". format(l)]["duplicates"]["dup_loc{0}".format(x+y+1)]
            lib["track{0}". format(prev_tr)]["duplicates"]["dup_loc{0}".format(x+y+1)] = a
    # print(Duplicate_Index)

    dup = lib["track{0}". format(l)]["duplicates"]["num_dup"]

    if dup >= 1:
        for i in range(dup+1):
            loc = lib["track{0}". format(l)]["duplicates"]["dup_loc{0}".format(i)]
            track_uri = lib["track{0}". format(loc)]["uri"]
            lib["track{0}". format(l)]["uri{0}".format(i)] = track_uri
    else:
        lib["track{0}". format(l)]["uri0"] = track_uri

    l = l+1

print(lib["track0"]["duplicates"])
print(lib["track1"]["duplicates"])
print(lib["track2"]["duplicates"])
print(lib["track3"]["duplicates"])
print(lib["track4"]["duplicates"])
print(lib["track5"]["duplicates"])
print(lib["track6"]["duplicates"])
print(lib["track7"]["duplicates"])
print(lib["track8"]["duplicates"])
print(lib["track9"]["duplicates"])
print(lib["track10"]["duplicates"])

The Downfall of Us All
The Downfall Of Us All
Adam's Song
Adam's Song
Adam's Song
Alive
Alive
Alive
Alive
Alive
On The Floor
{'dup_id': 0, 'num_dup': 1, 'dup_loc0': 0, 'dup_loc1': 1}
{'dup_id': 0, 'num_dup': 1, 'dup_loc0': 0, 'dup_loc1': 1}
{'dup_id': 2, 'num_dup': 2, 'dup_loc0': 2, 'dup_loc1': 3, 'dup_loc2': 4}
{'dup_id': 2, 'num_dup': 2, 'dup_loc0': 2, 'dup_loc1': 3, 'dup_loc2': 4}
{'dup_id': 2, 'num_dup': 2, 'dup_loc0': 2, 'dup_loc1': 3, 'dup_loc2': 4}
{'dup_id': 5, 'num_dup': 2, 'dup_loc0': 5, 'dup_loc1': 7, 'dup_loc2': 9}
{'dup_id': 6, 'num_dup': 1, 'dup_loc0': 6, 'dup_loc1': 8}
{'dup_id': 5, 'num_dup': 2, 'dup_loc0': 5, 'dup_loc1': 7, 'dup_loc2': 9}
{'dup_id': 6, 'num_dup': 1, 'dup_loc0': 6, 'dup_loc1': 8}
{'dup_id': 5, 'num_dup': 2, 'dup_loc0': 5, 'dup_loc1': 7, 'dup_loc2': 9}
{'dup_id': -1, 'num_dup': 0}


In [1686]:
# Number of [non-duplicate songs, duplicates of song 3, duplicates of song 5,...]
# num_uniques [#nd, #song3, #song5...]
num_uniques = []

# Labeling uniques and duplicates
# uniques = [-1, 1 , 2...]
uniques = list(set(Duplicate_Index))
uniques.sort()

for i in range (len(uniques)):
    count = Duplicate_Index.count(uniques[i])
    num_uniques.append(count)

print(Duplicate_Index)
print(uniques)
print(num_uniques)


[0, 0, 2, 2, 2, 5, 6, 5, 6, 5, -1]
[-1, 0, 2, 5, 6]
[1, 2, 3, 3, 2]


In [1687]:
# df = pd.DataFrame(lib)
df = pd.DataFrame.from_dict(lib, orient='index')
df2 = df['artists'].apply(pd.Series)
df3 = pd.concat([df.drop(['artists'], axis=1), df2], axis=1)


df3.to_csv(r'C:\Users\mackt\Python\Music Library\Data\Playlist.csv',index = False)

In [1688]:
# Sort Alphabetically by Artist -> Album -> Song
df = df.sort_values(['Artist 1', 'Album', 'Song'], ascending = True, key = lambda col: col.str.lower())

# Track IDs
sl_tr_id = list(range(1,len(sl_tr_name)+1))
df['ID'] = sl_tr_id

# Creating CSV of Entire Playlist
df.to_csv(r'C:\Users\mackt\Python\Music Library\Data\Playlist.csv',index = False)

df = df.sort_index()


KeyError: 'Artist 1'

In [ ]:
# Change the dates of all duplicate songs to the earliest release date
dup_date = {}

for i in range(len(Duplicate_Index)):
    if Duplicate_Index[i] != -1:
        if Duplicate_Index[i] in dup_date:
            if sl_tr_release[i] > dup_date[Duplicate_Index[i]]:
                sl_tr_release[i] = dup_date[Duplicate_Index[i]]
            else:
                dup_date[Duplicate_Index[i]] = sl_tr_release[i]
        else:
            dup_date[Duplicate_Index[i]] = sl_tr_release[i]
            dup_date["Loc{0}".format(Duplicate_Index[i])] = i
for i in range(len(Duplicate_Index)):
    if Duplicate_Index[i] != -1:
        sl_tr_release[i] = dup_date[Duplicate_Index[i]]
    # print(sl_tr_release[i])

df['Spot_Date'] = sl_tr_release

In [ ]:
# Dropping Duplicates
df = df.sort_values(['Popularity'], ascending = False)
df = df.drop_duplicates(subset = ['song', 'Artist 1', 'Artist 2', 'Artist 3', 'Artist 4', 'Artist 5'], keep = 'first')
df = df.sort_values(['Artist 1', 'Album', 'Song'], ascending = True, key = lambda col: col.str.lower())

df.drop(['Duplicates', 'song'], axis=1, inplace=True)

sl_tr_id = list(range(1,len(df)+1))
df['ID'] = sl_tr_id

# Creating CSV of Entire Playlist Without Duplicates
df.to_csv(r'C:\Users\mackt\Python\Music Library\Data\Playlist_No_Duplicates.csv',index = False)

In [ ]:
print(df)
